In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#default_exp classify_image

In [3]:
#export
from food.tools import *
from food.paths import *

import pandas as pd
import numpy as np
from food.psql import *

# !nbdev_build_lib
import requests
from food.clipmodel import image2clip
from PIL import Image

In [4]:
#export
from food.milvus import collection
collection.load()

In [5]:
#export
def search_by_clip(clip, topk=10, query_expr='',search_params = {'ef':10000}): #32768
    search_params = {"metric_type": "IP",
                     "params": search_params}

    print(query_expr)
    results = collection.search(clip[None,], "clip", search_params, limit=topk, expr=query_expr, output_fields=["id"])
    results = results[0]
    return list(results.distances), results.ids

def get_metadata(ids):
    q = f"""select * from foods_big where id in {tuple(ids)}"""
    return pd.read_sql(q,engine)

In [34]:
#export
def search_image(url=None,path=None):
    if url:
        response = requests.get(url, stream=True)
        image = Image.open(response.raw)
    elif path:
        image = Image.open(path)
    clip = image2clip(image)
    results = search_by_clip(clip.numpy())
    df = get_metadata(results[1])
    df['score'] = results[0]
    return df

In [22]:
#export
def get_calories(url=None,path=None):
    r = search_image(url=url)
    r = r[(r['energy_kcal_100g']>r['energy_kcal_100g'].quantile(0.3))&(r['energy_kcal_100g']<r['energy_kcal_100g'].quantile(0.9))]
    return r[['energy_kcal_100g','proteins_100g','fat_100g','carbohydrates_100g']].mean().round(2)

In [32]:
url = 'https://cdn.shopify.com/s/files/1/0943/7472/products/12540011_0_640x640_b3b5eedd-b502-4b22-8fb4-70c040c4798b_x700.jpg?v=1614184159'

In [35]:
search_image(url)

,id,product_name,keywords,ingredients_text,categories,food_groups,energy_kcal_100g,proteins_100g,fat_100g,carbohydrates_100g,clip,score
0,35861,"Lindt excellence, dark chocolate, dark","snack, chocolate, candie, confectionerie, exce...","Sugar, chocolate, cocoa butter, milkfat, soya ...","Snacks, Sweet snacks, Confectioneries, Chocola...",chocolate-products,475.0,5.0,32.500,60.000,"[0.006479646, 0.036235914, 0.029764645, 0.0477...",0.379934
1,35872,"Lindt, excellence, black currant dark chocolat...","bar, confectionerie, snack, dark, sweet, curra...","Sugar, chocolate, black currant pieces (black ...","Snacks, Sweet snacks, Confectioneries, Chocola...",sweets,500.0,5.0,30.000,57.500,"[0.013169993, 0.039979216, 0.051202036, 0.0358...",0.364851
2,113976,Schmerling's 85% Cocoa Extra Dark,"dark, cocoa, made-in-switzerland, schmerling, ...",NaN,NaN,NaN,587.0,15.0,43.000,20.000,"[0.028687201, 0.0069407155, 0.020639028, 0.004...",0.360589
3,227195,Lindt Excellence Dark Chocolate Bar - 70%,"ariel, snack, it, excellence, fine, vegan, bar...","cocoa mass, sugar, cocoa butter, natural bourb...","Snacks, Sweet snacks, Cocoa and its products, ...",chocolate-products,566.0,9.5,41.000,34.000,"[-0.0012109026, 0.03580227, 0.032108422, 0.031...",0.347456
4,227314,Lindt excellence 90% cacao,"noir, tablette, vert, derive, point, cacao, su...","pâte de cacao, beurre de cacao, cacao maigre, ...","Snacks,Snacks sucrés,Cacao et dérivés,Chocolat...",chocolate-products,592.0,10.0,55.000,14.000,"[0.0117876865, 0.039992243, 0.040229462, -0.01...",0.344328
5,458309,Lindt Tab Exc Dark 70% Cocoa Extra Thins 200G,"tab, dark, extra, cocoa, lindt, thin, 200g, ex...",NaN,NaN,NaN,567.0,9.5,41.000,34.000,"[0.0038524915, 0.05194572, 0.025168194, 0.0223...",0.335768
6,546920,Lindt Excellence Mild 85% Probierpreis -23%,"85, dark-chocolate, 23, probierprei, mild, exc...","Kakao: 85% mindestens Zutaten: Kakaomasse, Kak...","Snacks, Snacks sucrés, Cacao et dérivés, Choco...",chocolate-products,603.0,8.9,55.000,19.000,"[0.00069913967, 0.027529106, 0.02723215, 0.004...",0.334607
7,631486,Extra Dark 85% Chocolate,"sweet, cacao, sustainable, farming, extra, utz...","Cocoa Mass, Cocoa Butter, Sugar, Cocoa Powder,...","Snacks, Sweet snacks, Chocolates, Dark chocolates",chocolate-products,591.0,10.4,50.400,18.200,"[0.01028741, 0.033864982, 0.005958988, 0.02844...",0.334338
8,636991,Noir 85% Extra dunkle Schokolade,"dunkle, extra, 85, schokolade, spar, noir, pre...",NaN,NaN,NaN,599.0,10.0,50.000,18.000,"[0.017121695, 0.014120773, 0.0142110195, 0.007...",0.334109
9,670513,Lindt EXCELLENCE 95% cocoa,"95, snack, excellence, lindt, dark, sweet, cho...","cocoa mass, cocoa butter, cocoa powder, sugar,...","Snacks, Sweet snacks, Cocoa and its products, ...",chocolate-products,620.0,12.5,56.875,9.375,"[0.010039008, 0.040425982, 0.051311944, 0.0554...",0.334108


In [33]:
get_calories(url)

energy_kcal_100g      589.83
proteins_100g          10.63
fat_100g               49.07
carbohydrates_100g     20.53
dtype: float64

energy_kcal_100g      48.83
proteins_100g          0.63
fat_100g               0.05
carbohydrates_100g    11.40
dtype: float64

In [16]:
r

,id,product_name,keywords,ingredients_text,categories,food_groups,energy_kcal_100g,proteins_100g,fat_100g,carbohydrates_100g,clip,score
0,195675,Oranges (Navel) bio,"et, boisson, origine, de, europeen, espagne, d...",Oranges.,"Aliments et boissons à base de végétaux, Alime...",fruits,62.0,0.1,0.0,15.5,"[0.009044354, 0.03422737, -0.0042800787, 0.011...",0.286314
2,195689,Oranges à jus,"frai, marque, san, base, orange, aliment, legu...",Oranges à jus. Conservé au moyen de Thiabendaz...,"Aliments et boissons à base de végétaux, Alime...",fruits,47.0,0.9,0.0,12.0,"[0.012643795, 0.01661294, -0.01384048, 0.01231...",0.280501
4,260911,Oranges à déguster,"aliment, categorie, orange, legume, produit, f...",Oranges Navel Lane-Late. Revêtement : E473.,"Aliments et boissons à base de végétaux, Alime...",fruits,47.0,0.9,0.0,12.0,"[0.00019604036, 0.023238743, -0.0020484284, 0....",0.279437
5,304827,Oranges dégustation,"ab, agriculture, legume, produit, aliment, ora...",Oranges,"Aliments et boissons à base de végétaux, Alime...",fruits,47.0,1.0,0.2,8.3,"[-0.0060053943, 0.011059979, -0.0015331417, -0...",0.279147
6,394780,Re:Freshing Pomaranca Orange,"nectar, pomaranca, fructal, orange, re-freshing",,"Aliments et boissons à base de végétaux, Boiss...",fruit-nectars,47.0,0.3,0.0,11.2,"[0.02503221, 0.05851776, 0.0057875738, 7.84029...",0.278901
8,498778,Zumosol orange,"zumosol, orange",NaN,NaN,NaN,43.0,0.6,0.1,9.4,"[0.024359493, 0.0911041, -0.012610068, 0.01364...",0.278311


In [20]:
pd.read_sql("select count(*) from foods_big where energy_kcal_100g is null",engine)

,count
0,0


In [118]:
pd.read_sql("select * from foods_big where clip is not null",engine)

,id,product_name,keywords,ingredients_text,categories,food_groups,energy_kcal_100g,proteins_100g,fat_100g,carbohydrates_100g,clip


In [48]:
df = pd.read_sql("select * from foods_big",engine).set_index('id')

In [103]:
text = df[['product_name','keywords','ingredients_text','categories','food_groups']].fillna("")
for c in text.columns: text[c] = text[c].str.replace('NaN','')
text = text['product_name']+ '. ' + text['food_groups']+ '. ' + text['keywords'] + '. ' + text['ingredients_text'] + '. '+ text['categories']

In [117]:
text.loc[text.sample(1).index[0]]

"panna cotta fruits rouges. dairy-desserts. panna, de, au, produit, lacte, dessert, rouge, laitier, franprix, framboise, couli, la, fruit, cotta. Crème fraîche 31,7% - eau - sirop de glucose - framboises 10,4% - sucre - lait écrémé en poudre - huile de coco - alcool - amidon de tapioca - épaississants : carraghénanes, alginate de sodium - correcteur d’acidité : acide citrique - gélifiant : pectines - arômes naturels - émulsifiant : mono - et diglycérides d’acides gras - extrait de sureau - conservateur : sorbate de potassium.Teneur en alcool : 1,0%.Traces d'œuf et blé.Les informations en gras sont destinées aux personnes intolérantes ou allergiques.. Produits laitiers, Desserts, Desserts lactés, Panna cottas, Panna cottas au coulis de fruits, Panna cottas à la framboise"

In [36]:
!nbdev_build_lib

Converted 00_paths.ipynb.
Converted 00_progress_check.ipynb.
Converted 00_psql.ipynb.
Converted 00_tools.ipynb.
Converted 04_milvus.ipynb.
Converted 06_clip.ipynb.
Converted 0_template.ipynb.
Converted bot.ipynb.
Converted classfy_image.ipynb.
Converted inference.ipynb.
Converted milvus_update.ipynb.
Converted openfoodfacts_json_parsing.ipynb.
